# Double Basis
Mainly for debuging & testing

In [1]:
# Import package
using LinearAlgebra, Plots
DEV = true
if DEV    # use local package
    include("../src/EDKit.jl")
    using .EDKit
else      # use EDKit in the Pkg system
    using EDKit
end

## Projector
Consider a spin-1/2 translational invariant random Hamiltonian 
$$
H = \sum_i h_{i,i+1}
$$
The Hamiltonian is irreducible in the $k=0$ sector.

If we devided the Hilbert space further to $k=0, N=0,\dots,L$, the Hamiltonian will have off diagonal blocks $H_{mn}$. 

In [2]:
L = 10
h0 = rand(ComplexF64, 4, 4) |> Hermitian
bases = [ProjectedBasis(L=L, N=i) for i in 0:L]
blocks = Matrix{Any}(undef, L+1, L+1)
for m in 1:L+1, n in 1:L+1 
    basis = DoubleBasis(bases[m], bases[n])
    blocks[m,n] = trans_inv_operator(h0, 2, basis)
end

In above, each `block[m,n]` is the $H_{mn}$. We can glue the blocks together to check its correctness by examine the eigenvalues:

In [3]:
dims = [size(b, 1) for b in bases]
mat = zeros(ComplexF64, 2^L, 2^L)
n=0
for i = 1:L+1 
    m = 0
    for j = 1:L+1
        EDKit.addto!(view(mat, n+1:n+dims[i], m+1:m+dims[j]), blocks[i,j])
        m += dims[j]
    end
    n += dims[i]
end

# original matrix
mat0 = trans_inv_operator(h0, 2, L)

# Eigen values
vals0 = eigvals(Hermitian(mat0))
vals = eigvals(Hermitian(mat))
vals - vals0 |> norm

6.162088209991788e-13

## Transformation
Consider mapping a state in ($N,k$) sector to tensor-product basis, and then the reverse map from tensor basis back to translational basis.

In [4]:
printdgt!(B, i) = (change!(B, i); print("|"); print.(B.dgt); print("⟩; "))
function showinfo(;L, k, N=nothing, p=nothing)
    B0 = TensorBasis(L=L)
    B1 = if isnothing(N) 
        isnothing(p) ? TranslationalBasis(L=L, k=k) : TranslationParityBasis(L=L, k=k, p=p)
    else
        isnothing(p) ? TranslationalBasis(L=L, N=N, k=k) : TranslationParityBasis(L=L, N=N, k=k, p=p)
    end
    dim = size(B1, 1); println("Dimension = $dim")
    for x in 1:dim  # check all basis states in `B1`
        println("--------------------------------------------------")
        v = zeros(dim); v[x] = 1; print("Ψ$x = "); printdgt!(B1, x); print("\n")
        v0 = DoubleBasis(B0, B1)(v)  # Map to tensor basis

        print("Basis: ")  # Print the results
        angles = Int[]
        for i in eachindex(v0) 
            iszero(v0[i]) && continue
            printdgt!(B0, i)
            push!(angles, round(Int, angle(v0[i])*L/2/π) )
        end
        print("\nAngle: θ₀ * ", angles, "\n")
        println("Norm: $(norm(v0))")
    
        v1 = DoubleBasis(B1, B0)(v0)  # Map to translational basis 
        print("V1 = ")
        for i in eachindex(v1) 
            iszero(v1[i]) ? continue : print("$(v1[i]) * ")
            printdgt!(B1, i)
        end
        print("\n")
    end
end
showinfo(L=6, N=3, k=0)

Dimension = 4
--------------------------------------------------
Ψ1 = |000111⟩; 
Basis: |

000111⟩; |001110⟩; |011100⟩; |100011⟩; |110001⟩; |111000⟩; 
Angle: θ₀ * 

[0, 0, 0, 0, 0, 0]
Norm: 1.0
V1 = 1.0 * |000111⟩; 
--------------------------------------------------
Ψ2 = |001011⟩; 
Basis: |001011⟩; |010110⟩; |011001⟩; |100101⟩; |101100⟩; |110010⟩; 
Angle: θ₀ * [0, 0, 0, 0, 0, 0]
Norm: 1.0
V1 = 1.0 * |001011⟩; 
--------------------------------------------------
Ψ3 = |001101⟩; 
Basis: |001101⟩; |010011⟩; |011010⟩; |100110⟩; |101001⟩; |110100⟩; 
Angle: θ₀ * [0, 0, 0, 0, 0, 0]
Norm: 1.0
V1 = 1.0 * |001101⟩; 
--------------------------------------------------
Ψ4 = |010101⟩; 
Basis: |010101⟩; |101010⟩; 
Angle: θ₀ * [0, 0]
Norm: 1.0
V1 = 1.0 * |010101⟩; 


In [5]:
showinfo(L=6, N=3, k=1)

Dimension = 3
--------------------------------------------------
Ψ1 = |000111⟩; 
Basis: |000111⟩; |001110⟩; |011100⟩; |100011⟩; |110001⟩; |111000⟩; 
Angle: θ₀ * [0, -1, -2, 1, 2, -3]
Norm: 1.0000000000000002


V1 = 1.0 - 0.0im * |000111⟩; 
--------------------------------------------------
Ψ2 = |001011⟩; 
Basis: |001011⟩; |010110⟩; |011001⟩; |100101⟩; |101100⟩; |110010⟩; 
Angle: θ₀ * [0, -1, -3, 1, -2, 2]
Norm: 1.0000000000000002
V1 = 1.0 - 0.0im * |001011⟩; 
--------------------------------------------------
Ψ3 = |001101⟩; 
Basis: |001101⟩; |010011⟩; |011010⟩; |100110⟩; |101001⟩; |110100⟩; 
Angle: θ₀ * [0, 2, -1, 1, -3, -2]
Norm: 1.0
V1 = 1.0 + 0.0im * |001101⟩; 


In [6]:
showinfo(L=6, N=3, k=2)

Dimension = 3
--------------------------------------------------
Ψ1 = |000111⟩; 
Basis: |000111⟩; |001110⟩; |011100⟩; |100011⟩; |110001⟩; |111000⟩; 
Angle: θ₀ * [0, -2, 2, 2, -2, 0]
Norm: 1.0
V1 = 1.0 + 0.0im * |000111⟩; 
--------------------------------------------------
Ψ2 = |001011⟩; 
Basis: |001011⟩; |010110⟩; |011001⟩; |100101⟩; |101100⟩; |110010⟩; 
Angle: θ₀ * [0, -2, 0, 2, 2, -2]
Norm: 1.0000000000000002
V1 = 1.0 + 0.0im * |001011⟩; 
--------------------------------------------------
Ψ3 = |001101⟩; 
Basis: |001101⟩; |010011⟩; |011010⟩; |100110⟩; |101001⟩; |110100⟩; 
Angle: θ₀ * [0, -2, -2, 2, 0, 2]
Norm: 1.0000000000000002
V1 = 1.0 - 0.0im * |001101⟩; 


In [7]:
showinfo(L=6, N=3, k=3)

Dimension = 4
--------------------------------------------------
Ψ1 = |000111⟩; 
Basis: |000111⟩; |001110⟩; |011100⟩; |100011⟩; |110001⟩; |111000⟩; 
Angle: θ₀ * [0, 3, 0, 3, 0, 3]
Norm: 1.0
V1 = 1.0 * |000111⟩; 
--------------------------------------------------
Ψ2 = |001011⟩; 
Basis: |001011⟩; |010110⟩; |011001⟩; |100101⟩; |101100⟩; |110010⟩; 
Angle: θ₀ * [0, 3, 3, 3, 0, 0]
Norm: 1.0
V1 = 1.0 * |001011⟩; 
--------------------------------------------------
Ψ3 = |001101⟩; 
Basis: |001101⟩; |010011⟩; |011010⟩; |100110⟩; |101001⟩; |110100⟩; 
Angle: θ₀ * [0, 0, 3, 3, 3, 0]
Norm: 1.0
V1 = 1.0 * |001101⟩; 
--------------------------------------------------
Ψ4 = |010101⟩; 
Basis: |010101⟩; |101010⟩; 
Angle: θ₀ * [0, 3]
Norm: 1.0
V1 = 1.0 * |010101⟩; 


In [8]:
showinfo(L=4, k=0)

Dimension = 6
--------------------------------------------------
Ψ1 = |0000⟩; 
Basis: |0000⟩; 
Angle: θ₀ * [0]
Norm: 1.0
V1 = 1.0 * |0000⟩; 
--------------------------------------------------
Ψ2 = |0001⟩; 
Basis: |0001⟩; |0010⟩; |0100⟩; |1000⟩; 
Angle: θ₀ * [0, 0, 0, 0]
Norm: 1.0
V1 = 1.0 * |0001⟩; 
--------------------------------------------------
Ψ3 = |0011⟩; 
Basis: |0011⟩; |0110⟩; |1001⟩; |1100⟩; 
Angle: θ₀ * [0, 0, 0, 0]
Norm: 1.0
V1 = 1.0 * |0011⟩; 
--------------------------------------------------
Ψ4 = |0101⟩; 
Basis: |0101⟩; |1010⟩; 
Angle: θ₀ * [0, 0]
Norm: 0.9999999999999999
V1 = 1.0 * |0101⟩; 
--------------------------------------------------
Ψ5 = |0111⟩; 
Basis: |0111⟩; |1011⟩; |1101⟩; |1110⟩; 
Angle: θ₀ * [0, 0, 0, 0]
Norm: 1.0
V1 = 1.0 * |0111⟩; 
--------------------------------------------------
Ψ6 = |1111⟩; 
Basis: |1111⟩; 
Angle: θ₀ * [0]
Norm: 1.0
V1 = 1.0 * |1111⟩; 


In [9]:
showinfo(L=4, k=1)

Dimension = 3
--------------------------------------------------
Ψ1 = |0001⟩; 
Basis: |0001⟩; |0010⟩; |0100⟩; |1000⟩; 
Angle: θ₀ * [0, -1, -2, 1]
Norm: 1.0
V1 = 1.0 - 0.0im * |0001⟩; 
--------------------------------------------------
Ψ2 = |0011⟩; 
Basis: |0011⟩; |0110⟩; |1001⟩; |1100⟩; 
Angle: θ₀ * [0, -1, 1, -2]
Norm: 1.0
V1 = 1.0 - 0.0im * |0011⟩; 
--------------------------------------------------
Ψ3 = |0111⟩; 
Basis: |0111⟩; |1011⟩; |1101⟩; |1110⟩; 
Angle: θ₀ * [0, 1, -2, -1]
Norm: 1.0
V1 = 1.0 + 0.0im * |0111⟩; 


In [10]:
showinfo(L=4, k=2)

Dimension = 4
--------------------------------------------------
Ψ1 = |0001⟩; 
Basis: |0001⟩; |0010⟩; |0100⟩; |1000⟩; 
Angle: θ₀ * [0, 2, 0, 2]
Norm: 1.0
V1 = 1.0 * |0001⟩; 
--------------------------------------------------
Ψ2 = |0011⟩; 
Basis: |0011⟩; |0110⟩; |1001⟩; |1100⟩; 
Angle: θ₀ * [0, 2, 2, 0]
Norm: 1.0
V1 = 1.0 * |0011⟩; 
--------------------------------------------------
Ψ3 = |0101⟩; 
Basis: |0101⟩; |1010⟩; 
Angle: θ₀ * [0, 2]
Norm: 0.9999999999999999
V1 = 1.0 * |0101⟩; 
--------------------------------------------------
Ψ4 = |0111⟩; 
Basis: |0111⟩; |1011⟩; |1101⟩; |1110⟩; 
Angle: θ₀ * [0, 2, 0, 2]
Norm: 1.0
V1 = 1.0 * |0111⟩; 


Now consider the map from ($N,k,p$) sector to tensor-product basis, and the reverse thereof.

In [11]:
showinfo(L=6, N=3, k=0, p=1)

Dimension = 3
--------------------------------------------------
Ψ1 = |000111⟩; 
Basis: |000111⟩; |001110⟩; |011100⟩; |100011⟩; |110001⟩; |111000⟩; 
Angle: θ₀ * [0, 0, 0, 0, 0, 0]
Norm: 1.0
V1 = 1.0 * |000111⟩; 
--------------------------------------------------
Ψ2 = |001011⟩; 
Basis: |001011⟩; |001101⟩; |010011⟩; |010110⟩; |011001⟩; |011010⟩; |100101⟩; |100110⟩; |101001⟩; |101100⟩; |110010⟩; |110100⟩; 
Angle: θ₀ * [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Norm: 1.0
V1 = 1.0000000000000002 * |001011⟩; 
--------------------------------------------------
Ψ3 = |010101⟩; 
Basis: |010101⟩; |101010⟩; 
Angle: θ₀ * [0, 0]
Norm: 1.0
V1 = 1.0 * |010101⟩; 


In [12]:
showinfo(L=6, N=3, k=0, p=-1)

Dimension = 1
--------------------------------------------------
Ψ1 = |001011⟩; 
Basis: |001011⟩; |001101⟩; |010011⟩; |010110⟩; |011001⟩; |011010⟩; |100101⟩; |100110⟩; |101001⟩; |101100⟩; |110010⟩; |110100⟩; 
Angle: θ₀ * [0, 3, 3, 0, 0, 3, 0, 3, 3, 0, 0, 3]
Norm: 1.0
V1 = 1.0000000000000002 * |001011⟩; 
